In [173]:
import numpy as np
import pandas as pd

import OSMPythonTools as osmpt
from OSMPythonTools.nominatim import Nominatim
#from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

import osm2geojson
#import overpass
#import overpy

#api = overpy.Overpass()

# fetch all ways and nodes
#result = api.query("""
#    way(50.746,7.154,50.748,7.157) ["highway"];
#    (._;>;);
#    out body;
#    """)
#result
#from osm_runner import Runner  # pip install osm-runner

In [226]:
nominatim = Nominatim()

data_table = pd.read_excel("таблица для карты пример.xlsx", names=["Республика", "Город", "Класс"], header=None)
data_table

,Республика,Город,Класс
0,Республика Дагестан,с. Балхар,1
1,Республика Дагестан,с. Сулевкент,1
2,Республика Дагестан,с. Унцукуль,2
3,Республика Дагестан,с. Агвали,3
4,Республика Дагестан,Цумадинский район,3
5,Республика Дагестан,с. Ахты,3
6,Республика Дагестан,с. Батлаич,3
7,Республика Дагестан,г. Дагестанские Огни,3
8,Республика Дагестан,г. Дербент,3
9,Республика Дагестан,с. Дылым,3


In [232]:
import requests
import osm2geojson
import json

# https://lz4.overpass-api.de/api/interpreter
# https://z.overpass-api.de/api/interpreter
# https://overpass.openstreetmap.ru/api/interpreter


def getGeoJSON(osm_id, search_str):
    # Polygon request
    response = requests.get('https://overpass.openstreetmap.ru/api/interpreter?data=[out:xml][timeout:25];(' + search_str + ';);out geom;')
    string = response.text.replace("\n","")
    geojson = osm2geojson.xml2geojson(string)
    return geojson


def getPolygon(osm_id):
    #print("GET POLYGON FUNCTION")
    search_str = "rel(" + str(osm_id) + ")"
    geojson = getGeoJSON(osm_id, search_str)
    #print(geojson)
    
    if geojson["features"] == []:
        search_str = "way(" + str(osm_id) + ")"
        
        geojson = getGeoJSON(osm_id, search_str)
        #print(geojson)
        
    #print("---------------------")
           
    #json.dumps(geojson)
    
    if geojson["features"] != []:
        if search_str == "rel(" + str(osm_id) + ")":
            return geojson["features"][0]["geometry"]["coordinates"]
        else:
            return geojson["features"][0]["geometry"]["coordinates"]
    else:
        return []
    
    #overpass_api = overpass.API(endpoint="https://overpass.openstreetmap.ru/api/interpreter", timeout=600)
    #data = overpass_api.get(search_str, verbosity='geom')
    #print(data)
    
    #return data[0]["geometry"]["coordinates"]
    
def getPolygonRepublic(republic_name):
    # Find info (id among that) by town name
    town_json = nominatim.query([republic_name]).toJSON()
    
    # Check founded place
    if town_json == []:
        not_found_arr.append(town)
        return []
    else:
        osm_id = town_json[0]["osm_id"]
            
        for town_j in town_json:
            if town_j["class"] == "boundary":
                osm_id = town_j["osm_id"]
                break
    
    return getPolygon(osm_id)
    

## Get DataFrame with places lat and lon

In [233]:
republic_names = data_table["Республика"].to_numpy()
town_names     = data_table["Город"].to_numpy()
classes        = data_table["Класс"].to_numpy()

i = 0
not_found_arr = [] # Places, that not found

osm_id_arr = []

res_republic_names = []
res_republic_polygons = []

# Result table with polygon information
result_df = pd.DataFrame(columns=['Id', 'Republic', 'Place', 'Class', 'Lat', 'Lon', 'Polygon'])

for republic, town_name, class_n in zip(republic_names, town_names, classes):
    for town in town_name.split(';'):
        #print("Iter", i)
        
        # Find info (id among that) by town name
        town_json = nominatim.query([republic, town]).toJSON()
    
        # Check founded place
        if town_json == []:
            not_found_arr.append(town)
            break
        else:
            osm_id = town_json[0]["osm_id"]
            town_lat = town_json[0]["lat"]
            town_lon = town_json[0]["lon"]
            
            # Search place
            for town_j in town_json:
                if town_j["class"] == "place":
                    osm_id   = town_j["osm_id"]
                    town_lat = town_j["lat"]
                    town_lon = town_j["lon"]
                    break
            # Than search boundary     
            for town_j in town_json:
                if town_j["class"] == "boundary":
                    osm_id   = town_j["osm_id"]
                    town_lat = town_j["lat"]
                    town_lon = town_j["lon"]
                    break
            
            # Lastly search administative type
            for town_j in town_json:
                if town_j["type"] == "administrative" and (town_j["class"] == "place" or town_j["class"] == "boundary"):
                    osm_id   = town_j["osm_id"]
                    town_lat = town_j["lat"]
                    town_lon = town_j["lon"]
                    break
              
        if osm_id not in osm_id_arr:
            osm_id_arr.append(osm_id)
        else:
            result_df.loc[result_df["Id"] == osm_id, "Class"] += "," + str(class_n) 
            break
            
            
        # DEBUG print
        #for founded_town in town_json:
        #    print(founded_town)
        
        if republic not in res_republic_names:
            res_republic_names.append(republic)
            polygons = getPolygonRepublic(republic)
            
            # Swap coordinates
            for polygon in polygons:
                if polygon != [] and len(polygon[0]) == 2:
                    for point in polygon:
                        tmp      = point[0]
                        point[0] = point[1]
                        point[1] = tmp
                else:
                    for polygon_i in polygon:
                        for point in polygon_i:
                            tmp      = point[0]
                            point[0] = point[1]
                            point[1] = tmp
            
            res_republic_polygons.append(polygons)
            
        # Get polygon by id
        polygons = getPolygon(osm_id)
        #print(polygon)
        
        # Swap and check
        for polygon in polygons:
            if polygon != [] and len(polygon[0]) == 2:
                for point in polygon:
                    tmp = point[0]
                    point[0] = point[1]
                    point[1] = tmp
                    
                    # Colud be wrong, very big bounds
                    if (abs(point[0] - float(town_lat)) > 10) or (abs(point[1] - float(town_lon)) > 10):
                        polygon_i = []
                        break
            else:
                    for polygon_i in polygon:
                        for point in polygon_i:
                            tmp = point[0]
                            point[0] = point[1]
                            point[1] = tmp    
                            
                            # Colud be wrong, very big bounds
                            if (abs(point[0] - float(town_lat)) > 10) or (abs(point[1] - float(town_lon)) > 10):
                                polygon_i = []
                                break
            
        
        # Fill table
        result_df.loc[i] = [osm_id, republic, town, str(class_n), town_lat, town_lon, polygons]
        
        i += 1

if not_found_arr != []:
    print("Не найдены:")
    for town in not_found_arr:
        print(town)
else:
    print("OK")

OK


In [234]:
print(res_republic_names)

display(result_df)
#result_df.to_excel("PlacesLatLon.xlsx", startcol=-1)
#print(res_republic_polygons)
#496423823

['Республика Дагестан', 'Москва']


,Id,Republic,Place,Class,Lat,Lon,Polygon
0,169805104,Республика Дагестан,с. Балхар,1,42.280094,47.231457,"[[[42.2807924, 47.2228813], [42.2811908, 47.22..."
1,578366609,Республика Дагестан,с. Сулевкент,1,43.471985,46.753593,"[[[43.4699686, 46.7503194], [43.462227, 46.754..."
2,256056368,Республика Дагестан,с. Унцукуль,2,42.712723,46.789848,"[[[42.7060299, 46.7914563], [42.7048496, 46.79..."
3,120329901,Республика Дагестан,с. Агвали,3,42.539375,46.122169,"[[[42.5392654, 46.1189934], [42.5388152, 46.12..."
4,1858762,Республика Дагестан,Цумадинский район,3,42.449703650000004,46.03902919369907,"[[[[42.378317, 45.734701], [42.379221, 45.7356..."
5,7902958,Республика Дагестан,с. Ахты,3,41.462914,47.742794,"[[[[41.4437418, 47.7009205], [41.4445716, 47.7..."
6,767697275,Республика Дагестан,с. Батлаич,3,42.5556903,46.7552123,"[[[42.5596026, 46.7519139], [42.557846, 46.759..."
7,4623552,Республика Дагестан,г. Дагестанские Огни,3,42.1152434,48.1864201,"[[[[42.1331181, 48.1688178], [42.1334913, 48.1..."
8,4623553,Республика Дагестан,г. Дербент,3,42.057858,48.2887648,"[[[[42.1237418, 48.2238688], [42.125844, 48.22..."
9,494981896,Республика Дагестан,с. Дылым,3,43.0677247,46.6347976,"[[[43.0730294, 46.6155033], [43.07433, 46.6168..."


In [235]:
osm_info_txt_file = open("OpenStreetMapsInfo.txt", "w")

for place_class, lat, lon, polygon in zip(result_df["Class"], result_df["Lat"], result_df["Lon"], result_df["Polygon"]):
    osm_info_txt_file.write(str(place_class) + ":" + str(lat) + ":" + str(lon) + ":" + str(polygon) + "\n")
    
osm_info_txt_file.write("|")
    
for polygon in res_republic_polygons:
    osm_info_txt_file.write(str(polygon) + "\n")
    
osm_info_txt_file.close()

## Put poligons into Yandex Maps

In [106]:
#print(data)
from IPython.display import display
from IPython.display import HTML
from IPython.display import Javascript

display(HTML(filename="Polygons_draw.html"))
#Javascript(filename="Polygons_draw.js")

## Support part

In [5]:
api = overpass.API(endpoint="https://overpass.openstreetmap.ru/api/interpreter", timeout=600)
data = api.get('relation(id:1858757);way(r);', verbosity='geom')
data

{"features": [{"geometry": {"coordinates": [[46.760249, 41.862527], [46.758493, 41.862551]], "type": "LineString"}, "id": 137991374, "properties": {"admin_level": "2", "border_type": "nation", "boundary": "administrative", "name": "Border Azerbaijan - Russia", "name:ru": "\u0410\u0437\u0435\u0440\u0431\u0430\u0439\u0434\u0436\u0430\u043d - \u0420\u043e\u0441\u0441\u0438\u044f", "natural": "ridge", "source": "www.eastgeology.com"}, "type": "Feature"}, {"geometry": {"coordinates": [[46.270298, 41.982971], [46.268596, 41.979343], [46.266885, 41.975952], [46.266048, 41.974559], [46.266151, 41.972386], [46.267753, 41.970219], [46.26999, 41.968667], [46.271428, 41.967504], [46.272255, 41.966577], [46.273944, 41.9665], [46.275469, 41.965852], [46.277859, 41.962384], [46.278757, 41.961169], [46.278804, 41.959677], [46.279474, 41.95858], [46.280956, 41.95581], [46.283204, 41.952974], [46.283797, 41.951877], [46.284083, 41.950784], [46.283755, 41.949784], [46.282409, 41.947791], [46.281459, 41.9